In [80]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [70]:
# Function of the drift diffusion model equation
def perfect_integrator(x, dt, tau, mu, sigma_i, xi_i, sigma_s, xi_s, n_trials, stimulus_duration, frames_duration):
    duration_stimulus = stimulus_duration/dt
    duration_frames = int(frames_duration/dt)
    stimulus_list = np.random.randn(n_trials, int(duration_stimulus/duration_frames))
    decision_value_list = []
    for stimulus in stimulus_list:
            # The starting decision value will be 0:
            x = 0
            decision_value = [x]
            # The process will be repeated a two hundred times (1 second stimulus)
            for xi_s in stimulus:
                for i in range(duration_frames):
                    xi_i = np.random.randn(1)
                    # Solved differential diffusion equation:
                    x = x - (dt/tau)*(-mu)+np.sqrt(dt/tau)*(sigma_i*xi_i+sigma_s*xi_s)
                    # All x will be stored in the deccision value list:
                    decision_value.append(x)
            decision_value_list.append(decision_value)
    return decision_value_list, stimulus_list

In [111]:
def DDM_absorbing(x, dt, tau, mu, sigma_i, xi_i, sigma_s, xi_s, n_trials, stimulus_duration, frames_duration,bound):
    duration_stimulus = stimulus_duration/dt
    duration_frames = int(frames_duration/dt)
    stimulus_list = np.random.randn(n_trials, int(duration_stimulus/duration_frames))
    decision_value_list = []
    for stimulus in stimulus_list:
            # The starting decision value will be 0:
            x = 0
            decision_value = [x]
            # The process will be repeated a two hundred times (1 second stimulus)
            for xi_s in stimulus:
                for i in range(duration_frames):
                    xi_i = np.random.randn(1)
                    # Solved differential diffusion equation:
                    x = x - (dt/tau)*(-mu)+np.sqrt(dt/tau)*(sigma_i*xi_i+sigma_s*xi_s)
                    if x >= bound:
                        if len(decision_value)<= duration_stimulus:
                            reached_bound = True
                            for i in range(duration_stimulus-len(decision_value)):
                                decision_value.append(bound)
                    if x <= -bound:
                        if len(decision_value) <= duration_stimulus:
                            reached_bound = True
                            for i in range(duration_stimulus-len(decision_value)):
                                decision_value.append(-bound)
                    else:
                        if len(decision_value) < duration_stimulus:
                            # All x before reaching the bound value will be stored in the deccision value list:
                            decision_value.append(x)
                    # All x will be stored in the deccision value list:
            decision_value_list.append(decision_value)
    return decision_value_list, stimulus_list
    

In [110]:
# Function of the drift diffusion model equation
def DDM_reflecting(x, dt, tau, mu, sigma_i, xi_i, sigma_s, xi_s, n_trials, stimulus_duration, frames_duration,bound):
    duration_stimulus = stimulus_duration/dt
    duration_frames = int(frames_duration/dt)
    stimulus_list = np.random.randn(n_trials, int(duration_stimulus/duration_frames))
    decision_value_list = []
    for stimulus in stimulus_list:
            # The starting decision value will be 0:
            x = 0
            decision_value = [x]
            # The process will be repeated a two hundred times (1 second stimulus)
            for xi_s in stimulus:
                for i in range(duration_frames):
                    xi_i = np.random.randn(1)
                    # Solved differential diffusion equation:
                    x = x - (dt/tau)*(-mu)+np.sqrt(dt/tau)*(sigma_i*xi_i+sigma_s*xi_s)
                    if x >= bound:
                        x = bound
                    elif x <= -bound:
                        x = -bound
                    # All x will be stored in the deccision value list:
                    decision_value.append(x)
            decision_value_list.append(decision_value)
    return decision_value_list, stimulus_list

In [109]:
# Function of the double well model
def DW(x, dt, tau, mu, sigma_i, xi_i, sigma_s, xi_s, alfa, n_trials, stimulus_duration, frames_duration):
    duration_stimulus = stimulus_duration/dt
    duration_frames = int(frames_duration/dt)
    stimulus_list = np.random.randn(n_trials, int(duration_stimulus/duration_frames))
    decision_value_list = []
    for stimulus in stimulus_list:
            # The starting decision value will be 0:
            x = 0
            decision_value = [x]
            # The process will be repeated a two hundred times (1 second stimulus)
            for xi_s in stimulus:
                for i in range(duration_frames):
                    xi_i = np.random.randn(1)
                    # Solved differential diffusion equation:
                    x = x -(dt/tau)*(-mu-alfa*x*2+4*x**3)+np.sqrt(dt/tau)*(sigma_i*xi_i+sigma_s*xi_s)
                    # All x will be stored in the deccision value list:
                    decision_value.append(x)
            decision_value_list.append(decision_value)
    return decision_value_list, stimulus_list

In [74]:
def accuracy(decision_value_list,mu):
    error_list = []
    for decision_value in decision_value_list:
        right_decision =  decision_value[len(decision_value)-1] >= 0
        right_mu = mu >= 0
        left_decision = decision_value[len(decision_value)-1] < 0
        left_mu = mu < 0
        if right_decision == right_mu or left_decision == left_mu:
            error_list.append(0)
        else:
            error_list.append(1)
    per_error = 100-(sum(error_list)/len(error_list)*100)
    return per_error

In [90]:
def kernel(decision_value_list, stimulus_list):
    binary_decision = []
    for decision_value in decision_value_list:
        if decision_value[len(decision_value)-1] >= 0:
            binary_decision.append(1)
        else:
            binary_decision.append(0)
    logit_mod = sm.Logit(binary_decision, stimulus_list)
    result = logit_mod.fit()
    pars =result.params
    return pars

In [ ]:
def plot():
        plt.plot(np.arange(0,len(variable),1),variable,label = round(labeling,2))           
        plt.title(title)
        plt.xlabel(xlabel)
        positions = [200/8*n for n in range(9)]
        labels = [1000/8*n for n in range(9)]
        plt.xticks(positions, labels)
        plt.ylabel("Decision value")
        plt.legend()